In [1]:
!pip install cplex
!pip install docplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43.3 MB 4.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 610 kB 25.6 MB/s 
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662847 sha256=b2d7b8ea31a6c05856bca4e1bae32fafc742b24147f38f7e234be3036026ff18
  Stored in directory: /root/.cache/pip/wheels/a7/c9/fb/cee5a89f304e77a39c466e625ac2830434b76eb8384999d116
Successfully built docplex


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import docplex.mp.model as md
from scipy.spatial import Delaunay
import random
import math
from itertools import permutations

**3 Partition Problem (NP-Complete problem)**

**Goal:** Partition $3n$ integers into $n$ $3-$element subsets all with the same sum.

Choose $3*n*n$ qubits, one for each element and one for each subset.

$$
H= A \underbrace{\sum_{i=1}^n \left[\sum_{j=1}^{3n} x_{ij} -3 \right]^2}_{\text{each subset has 3 elements}} + B \underbrace{\sum_{j=1}^{3n} \left[\sum_{i=1}^n x_{ij} -1 \right]^2}_{\text{each element assigned to one subset}} + C \sum_{i=1}^{n} \left[ \sum_{j=1}^{3n} n_jx_{ij} - \frac{1}{n} \sum_{k=1}^{3n} n_k \right]^2$$


---------

Note: Run this first segment of code as many times as does not admit an error so that the set $S$ has no empty sets or duplicate subsets.

In [83]:
np.random.seed(0)

In [84]:
n=3
p=3
m,A,B,C=p*n,100000,100000,1
numbers = np.random.randint(1,100,size=m)
k=np.max(numbers)
numbers=numbers/k

In [85]:
model = md.Model(name='3_partition',parameters={})
x = np.array(model.binary_var_list(m*n)).reshape((n,m))
H1 = A*np.sum([ [ (np.sum(x[i,:])-p)**2 ] for i in range(n)])
H2 = B*np.sum([ [ (np.sum(x[:,j])-1)**2 ] for j in range(m)])
H3 = C*np.sum([ np.sum([[numbers[j]*x[i,j] - np.sum(numbers)/n ] for j in range(m)])**2 for i in range(n) ])
H=H1+H2+H3
model.minimize(H)
solution = np.array(model.solve().get_value_list(x.flatten())).reshape(n,m)
print(solution)

[[0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 1.]]


In [86]:
groups=[]
numbers=numbers*k
for i in range(n):
  for j in range(m):
    groups.append([])
    if solution[i,j]==1:
      groups[i].append(numbers[j])
for i in range(n):
  print('Group '+str(i+1)+' = '+str(groups[i]))
print()
for i in range(n):
  print('Group '+str(i+1)+' sum is '+str(np.sum(groups[i]))+'.')

Group 1 = [68.0, 68.0, 10.0]
Group 2 = [45.0, 84.0, 22.0]
Group 3 = [48.0, 65.0, 37.0]

Group 1 sum is 146.0.
Group 2 sum is 151.0.
Group 3 sum is 150.0.


In [7]:
#vars(model)

## Range n Qubit Solution ($3n$ qubits numbered 1 to $n$)

Choose $3*n$ qubits, one for each element.

$$
H= A \underbrace{\left[\sum_{i=1}^{3n} x_{i} - p\frac{n(n+1)}{2} \right]^2}_{\text{each subset has 3 elements}} + B \underbrace{\left[\prod_{i=1}^{3n} x_{i}  - (n!)^p \right]^2}_{\text{each element assigned to one subset}} - C \underbrace{\sum_{i=1}^{3n} \left[ \sum_{i\neq j \neq l=2}^{3n} n_ix_{i} + n_j x_j + n_l x_l - \frac{1}{n} \sum_{k=1}^{3n} n_k (x_i+x_j-x_l) \right]^2}_{\text{minimizes by choosing one $x_i$ and trying to match two other spins to equal the mean of the total sum}} +  D \underbrace{\sum_{i \neq j \neq l =1}^{3n} T_{i,j,l} (x_i - x_j - x_l)^2}_{\text{minimizes by disjoining like numbers (i.e. large numbers wont be paired)}}$$

Add $T_{i,j,l}-np.sum(numbers)/n$ to last term?

In [87]:
A,C,D=10000,10,1
k=np.max(numbers)
T=np.empty((m,m,m))
for i in range(m):
  for j in range(m):
    for l in range(m):
      T[i,j,l]=numbers[i]+numbers[j]+numbers[l]
T=T/np.max(T)
numbers=numbers/k

In [91]:
model_2 = md.Model(name='3_partition_n',parameters={})
x= np.array(model_2.integer_var_list(m,lb=1,ub=n,name='partition'))
H1 = A*(np.sum(x)-p*n*(n+1)/2)**2
H2 = C*np.sum([ np.sum( [ np.sum([ [numbers[i]*x[i]+numbers[j]*x[j]+numbers[l]*x[l]-np.sum(numbers)/n*(x[i]+x[j]-x[l]) ] for l in range(m) if i!=j!=l ]) for j in range(m) ])**2 for i in range(m) ])
H3 = D*np.sum([ [ T[i,j,l]*(x[i]-x[j]-x[l])**2 ] for l in range(m) for j in range(m)  for i in range(m) if i!=j!=l ])
H=H1-H2+H3
model_2.minimize(H)
solution = np.array(model_2.solve().get_value_list(x.flatten()))
print(solution)

[3. 1. 1. 1. 1. 3. 1. 3. 3.]


In [92]:
groups=[]
numbers=numbers*k
for i in range(n):
  groups.append([])
  for j in range(m):
      if solution[j]==i+1:
        groups[i].append(numbers[j])

print('Groups: ',groups)
for i in range(n):
  print('Group '+str(i+1)+' sum is '+str(np.sum(groups[i]))+'.')

Groups:  [[4032.0, 5460.0, 5712.0, 5712.0, 7056.0], [], [3780.0, 840.0, 1848.0, 3108.0]]
Group 1 sum is 27972.0.
Group 2 sum is 0.0.
Group 3 sum is 9576.0.
